In [2]:
import os, sys, gc
import glob
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

train_paths = glob.glob('移动广告营销场景下的人群召回算法挑战赛公开数据/train_dataset/*/*.orc')
train_paths.sort()

test_paths = glob.glob('移动广告营销场景下的人群召回算法挑战赛公开数据/test_dataset/*/*/*.orc')
test_paths.sort()

image_paths = glob.glob('移动广告营销场景下的人群召回算法挑战赛公开数据/image_byte/*.txt')
image_paths.sort()
image_paths = [os.path.basename(x)[:-4] for x in image_paths]

In [20]:
def load_orc(path, train=True):
    if train:
        df = pd.read_orc(path)[['uid', 'creative_image_md5', 'click']]
        df = df[df['creative_image_md5'] != 'na']
        df['creative_image_md5'] = df['creative_image_md5'].apply(lambda x: image_paths.index(x))
        df['click'] = df['click'].astype(np.bool_)
        
        df = pd.concat([
            df[df['click']],
            df[~df['click']].sample(5000)
        ], axis=0)
    else:
        df = pd.read_orc(path)[['uid', 'creative_image_md5']]
        df = df[df['creative_image_md5'] != 'na']
        df['creative_image_encode'] = df['creative_image_md5'].apply(lambda x: image_paths.index(x))
        
    gc.collect()
    
    return df

In [12]:
# test_df = [load_orc(x, False) for x in tqdm_notebook(test_paths[:])]
train_df = [load_orc(x, True) for x in tqdm_notebook(train_paths[:])]

/tmp/ipykernel_28790/1882801668.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  train_df = [load_orc(x, True) for x in tqdm_notebook(train_paths[:])]


  0%|          | 0/149 [00:00<?, ?it/s]

In [14]:
train_df[0].shape

(50235, 3)

In [8]:
import pandas as pd

from surprise import Dataset, NormalPredictor, Reader, SVD, SVDpp
from surprise.model_selection import cross_validate

reader = Reader(rating_scale=(0, 1))
data = Dataset.load_from_df(pd.concat(train_df[:50], 0), reader)
print(cross_validate(SVDpp(n_epochs=2), data, cv=5))

/tmp/ipykernel_10945/2519294989.py:7: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  data = Dataset.load_from_df(pd.concat(train_df[:50], 0), reader)


{'test_rmse': array([0.23058688, 0.23307187, 0.2325818 , 0.23178389, 0.23299759]), 'test_mae': array([0.11072786, 0.1115454 , 0.10984755, 0.10947936, 0.11143533]), 'fit_time': (1.643669843673706, 1.6307125091552734, 1.664658546447754, 1.6445331573486328, 1.6656889915466309), 'test_time': (0.6042773723602295, 0.507967472076416, 0.3289833068847656, 0.3328075408935547, 0.3662583827972412)}


In [9]:
train_df_full = pd.concat(train_df, 0)

/tmp/ipykernel_10945/3910771530.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  train_df_full = pd.concat(train_df, 0)


In [10]:
del train_df
gc.collect()

24

In [11]:
data = Dataset.load_from_df(train_df_full, reader)
train_data = data.build_full_trainset()
algo = SVDpp(n_epochs=2)
algo.fit(train_data)

In [13]:
algo.predict('45762638', '584')

Prediction(uid='45762638', iid='584', r_ui=None, est=0.0900224460996544, details={'was_impossible': False})

In [17]:
!mkdir -p submit

In [40]:
with open('submit/submission_2023-05-14.csv', 'w') as up:
    up.write('uid,creative_image_md5,pro\n')
    for path in test_paths[:10]:
        df = load_orc(path, False)

        for row in tqdm_notebook(df.iterrows(), total=len(df)):
            pred = algo.predict(str(row[1].uid), str(row[1].creative_image_encode)).est
            pred = str(pred)[:8]
            up.write(f'{row[1].uid},{row[1].creative_image_md5},{pred}\n')

with open('submit/submission_2023-05-15.csv', 'w') as up:
    up.write('uid,creative_image_md5,pro\n')
    for path in test_paths[10:20]:
        df = load_orc(path, False)

        for row in tqdm_notebook(df.iterrows(), total=len(df)):
            pred = algo.predict(str(row[1].uid), str(row[1].creative_image_encode)).est
            pred = str(pred)[:8]
            up.write(f'{row[1].uid},{row[1].creative_image_md5},{pred}\n')

            
with open('submit/submission_2023-05-16.csv', 'w') as up:
    up.write('uid,creative_image_md5,pro\n')
    for path in test_paths[20:]:
        df = load_orc(path, False)

        for row in tqdm_notebook(df.iterrows(), total=len(df)):
            pred = algo.predict(str(row[1].uid), str(row[1].creative_image_encode)).est
            pred = str(pred)[:8]
            up.write(f'{row[1].uid},{row[1].creative_image_md5},{pred}\n')


/tmp/ipykernel_10945/3957378726.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for row in tqdm_notebook(df.iterrows(), total=len(df)):


  0%|          | 0/368969 [00:00<?, ?it/s]

  0%|          | 0/369850 [00:00<?, ?it/s]

  0%|          | 0/369389 [00:00<?, ?it/s]

  0%|          | 0/369151 [00:00<?, ?it/s]

...

  0%|          | 0/456957 [00:00<?, ?it/s]

  0%|          | 0/457697 [00:00<?, ?it/s]

  0%|          | 0/457431 [00:00<?, ?it/s]

  0%|          | 0/457352 [00:00<?, ?it/s]

  0%|          | 0/457822 [00:00<?, ?it/s]

In [41]:
!zip -r submit.zip submit

  adding: submit/ (stored 0%)
  adding: submit/submission_2023-05-14.csv (deflated 84%)
  adding: submit/submission_2023-05-16.csv (deflated 85%)
  adding: submit/submission_2023-05-15.csv (deflated 84%)
